In [54]:
import sys

sys.path.insert(1, "/Users/minhpham/Documents/Work/Python/Qiskit/New Project/FrozenYoghourt/")

from FrozenYoghourt import *
from FrozenYoghourt import *
from FrozenYoghourt.mode import *
from FrozenYoghourt.gates import *
from FrozenYoghourt.gates.single import *
from FrozenYoghourt.gates.double import *
from FrozenYoghourt.gates.multi import *
from FrozenYoghourt.maths import *
from FrozenYoghourt.quantum import *
from FrozenYoghourt.quantum.decomposition import *
from FrozenYoghourt.circuit import *
from FrozenYoghourt.visualization import *

import numpy as np
import sys

sys.path.insert(1, "/Users/minhpham/Documents/Research/laughing-umbrella/xx_synthesis/monodromy")

import monodromy

from monodromy.coordinates import unitary_to_monodromy_coordinate, monodromy_to_positive_canonical_coordinate

/Users/minhpham/Documents/Research/laughing-umbrella/xx_synthesis/monodromy/lrslib-071b/lrs


KAK decomposition gives us 

$$CAN(a_1, a_2, 0) \cdot (Z_d \otimes Z_e) \cdot (XX_\beta) = (Z_r \otimes Z_s) \cdot CAN(b_1, b_2, 0) \cdot (Z_t \otimes Z_u).$$

We want to equate these parameters to each other. $\textbf{Note:}$ We need to multiply everything by 2 to match the convention of the paper. We also need to reverse the order of the circuit

We now build numerical circuit to check the correctness of the inequalities of Theorem 5.3. Note that we have $a_1+a_2 \leq \frac{\pi}{2}$, $a_1 \geq a_2$ and $0 \leq \beta \leq \frac{\pi}{4}$.

Once we have set up our left hand side circuit, we now compute the canonical coordinates for the right hand side circuit

In [109]:
def generate_random_a1a2():
    # Generate a random value for y in the range [0, pi/4]
    a2 = np.random.uniform(0, np.pi / 4)
    
    # Generate a random value for x in the range [y, pi/2 - y]
    a1 = np.random.uniform(a2, np.pi / 2 - a2)
    
    assert a1+a2 <= np.pi/2, "a1 + a2 is greater than pi/2"
    assert a1>=a2, "a2 is greater than a1"
    assert a2 >=0, "a2 is greater than 0"
    
    return a1, a2

def canonical_coordinate(mat):
    # Compute canonical coordinate
    return monodromy_to_positive_canonical_coordinate(*unitary_to_monodromy_coordinate(mat)[:3])

In [113]:
# Left hand side circuit
Mode.toggle("n")

fail = {"b3=0": 0, "b1+b2≥": 0, "b1+b2≤": 0, "b1-b2≥": 0, "b1-b2≤": 0}
rounds = 1000

for _ in range(rounds):

    a1, a2, d, e, beta = *generate_random_a1a2(), *np.random.uniform(0, np.pi, 2), np.random.uniform(0, np.pi/4)

    xx_lhs = NumericalCircuit(2)

    xx_lhs.can(2*a1, 2*a2, 0)
    xx_lhs.rz([2*d, 2*e], [0, 1])
    xx_lhs.rxx(2*beta)


    b1, b2, b3 = canonical_coordinate(xx_lhs.unitary)

    try:
        assert np.isclose(b3, 0), "The z coordinate does not vanish"
    except:
        fail["b3=0"] +=1
    
    try:
        assert b1+b2 >= a1+a2 - beta, "First inequality not satisfied"
    except:
        fail["b1+b2≥"] +=1

    try:
        assert b1+b2 <= np.pi/2- np.abs(np.pi/2-(a1+a2+beta)), "Second inequality not satisfied"
    except:
        fail["b1+b2≤"] += 1
        
    try:
        assert b1-b2 >= np.abs(a1-a2-beta), "Third inequality not satisfied"
    except:
        fail["b1-b2≥"] +=1
    
    try:
        assert b1-b2 <= a1-a2+beta, "Fourth inequality not satisfied"
    except:
        fail["b1-b2≤"] +=1

NUMERICAL activated


In [114]:
fail

{'b3=0': 0, 'b1+b2≥': 84, 'b1+b2≤': 133, 'b1-b2≥': 101, 'b1-b2≤': 117}